In [3]:
import os
import sys
import librosa
import tflearn
import tensorflow as tf
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
%matplotlib inline

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
os.chdir('C:/Users/karti/Speech/LibriSpeech/dev-clean/')

In [5]:
new_y=[]
f = open("y.txt", "r")
for x in f:
  new_y.append(int(x.strip()))

In [6]:
new_y = np.array(new_y).reshape(-1,1)

In [7]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(new_y)
res=enc.transform(new_y).toarray()

In [8]:
res

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [9]:
import pickle
new_mfccs_padded_gauss=[]
with open('padding_gaussian_mfccs_flatten.pkl', 'rb') as f:
    new_mfccs_padded_gauss = pickle.load(f)

In [10]:
len(new_mfccs_padded_gauss)

2703

In [11]:
len(new_mfccs_padded_gauss[0])

9542

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_mfccs_padded_gauss, res, test_size=0.33, random_state=42)

In [13]:
net = tflearn.input_data(shape=[None, 9542]) #Two wave chunks
net = tflearn.fully_connected(net, 1024, activation='relu')
net = tflearn.dropout(net, 0.4)
net = tflearn.fully_connected(net, 512, activation='relu')
net = tflearn.dropout(net, 0.35)
net = tflearn.fully_connected(net, 256, activation='relu')
net = tflearn.dropout(net, 0.3)
net = tflearn.fully_connected(net, 128, activation='relu')
net = tflearn.dropout(net, 0.25)
net = tflearn.fully_connected(net, 64, activation='relu')
net = tflearn.dropout(net, 0.2)
net = tflearn.fully_connected(net, 40, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
model = tflearn.DNN(net)

Instructions for updating:
Use tf.cast instead.


In [15]:
model.fit(X_train, y_train, n_epoch=1000, show_metric=True, batch_size=128,snapshot_step=100)

Training Step: 14999  | total loss: 3.62582 | time: 0.772s
| Adam | epoch: 1000 | loss: 3.62582 - acc: 0.0636 -- iter: 1792/1811
Training Step: 15000  | total loss: 3.62246 | time: 0.830s
| Adam | epoch: 1000 | loss: 3.62246 - acc: 0.0604 -- iter: 1811/1811
--


In [16]:
y_pred = model.predict(X_train)

In [17]:
np.sum(np.argmax(y_train, axis=1)==np.argmax(y_pred, axis=1))/len(y_train)

0.07730535615681944

In [18]:
y_pred = model.predict(X_test)

In [19]:
np.sum(np.argmax(y_test, axis=1)==np.argmax(y_pred, axis=1))/len(y_test)

0.05044843049327354

In [ ]:
model.save('model_padded_gauss.tflearn')

# Reload Saved Model

In [4]:
net = tflearn.input_data(shape=[None, 13]) #Two wave chunks
net = tflearn.fully_connected(net, 256, activation='relu')
net = tflearn.dropout(net, 0.4)
net = tflearn.fully_connected(net, 128, activation='relu')
net = tflearn.dropout(net, 0.25)
net = tflearn.fully_connected(net, 64, activation='relu')
net = tflearn.dropout(net, 0.2)
net = tflearn.fully_connected(net, 40, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')
model = tflearn.DNN(net)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\karti\Speech\LibriSpeech\dev-clean\model_mean.tflearn


In [97]:
model.load('model_padded_gauss.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\karti\Speech\LibriSpeech\dev-clean\model_mean.tflearn
